In [ ]:
from __future__ import division
from os.path import isdir, join, isfile, sep
from os import listdir 
from functools import partial
from warnings import warn
import numpy as np
from research_pyutils.path_related_functions import mkdir_p, rm_if_exists
import logging
from pathlib import Path
from workerbee import exhaust_all_files_randomly

# menpo packages imports
import menpo.io as mio
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpodetect.dlib.conversion import pointgraph_to_rect
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup

In [ ]:
path_base = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/300vw_testset/'
assert(isdir(path_base))

path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

In [ ]:
# def return_detector(detection):
#     if detection == 'dlib':
#         return load_dlib_frontal_face_detector()
#     elif detection == 'opencv':
#         from functools import partial
#         det = load_opencv_frontal_face_detector()
#         return partial(det, min_neighbours=3)
#     elif detection == 'ffld2':
#         return load_ffld2_frontal_face_detector()
#     else:
#         raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))

def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

In [ ]:
method_detection = 'opencv'
method_landm_loc = 'dlibERT'

fold_out = 'detection_' + method_detection + '_' + method_landm_loc + '_init_from_previous'
save_bb = 1  # if set to 0, then it won't save the bb.
fold_det_out = 'detector_' + method_detection + '_init_from_previous'
# detector = return_detector(method_detection)

In [ ]:
if method_landm_loc == 'dlibERT':
    path_shape_pred = path_pickles + 'modelln_' +  method_detection + '_' + method_landm_loc + '.model'
    assert(isfile(path_shape_pred))
    predictor_dlib = shape_predictor(path_shape_pred)

In [ ]:
def _get_path(p0, name):
    assert(isdir(p0))
    p1 = join(p0, name, '')
    if not isdir(p1):
        mkdir_p(p1)
    return p1
    
def process_frame(p_fr):
    # using global: detector(), p_ln_out_0, model, p_condor_dummy_0, p_bb_out_0, save_bb, predictor_dlib()
    try:
        im = mio.import_image(p_fr)
        if im.n_channels == 3:
            im = im.as_greyscale()
        p_bb_out = _get_path(p_bb_out_0, im.path.parent.name) + im.path.stem + '.pts'
        if isfile(p_bb_out):  # for this version, bb is loaded instead of detected
            ln = mio.import_landmark_file(p_bb_out)
            assert(ln.lms.n_points == 4)
#         detector(im)
#         assert(im.landmarks.n_groups < 10)  # might get complicated otherwise with the naming
#         if im.landmarks.n_groups > 0:  # there are detections
#             ln = im.landmarks[im.landmarks.group_labels[0]]
            
#             if save_bb:
#                 p_bb_out = _get_path(p_bb_out_0, im.path.parent.name) + im.path.stem + '.pts'
#                 mio.export_landmark_file(ln, p_bb_out, overwrite=True)

            p_out = _get_path(p_ln_out_0, im.path.parent.name) + im.path.stem + '.pts'

            if method_landm_loc == 'dlibERT':
                im_pili = np.array(im.as_PILImage())
                det_frame = predictor_dlib(im_pili, pointgraph_to_rect(ln.lms))
                init_pc = detection_to_pointgraph(det_frame)
                mio.export_landmark_file(LandmarkGroup.init_with_all_label(init_pc), 
                                         p_out, overwrite=True)
            else:
                ft = model.fit_from_bb(im, ln.lms)
                mio.export_landmark_file(ft.fitted_image['final'], p_out, overwrite=True)
        else:  # temp, can be removed, debugging purposes. 
            print("The input '{}' has no ln.".format(p_fr))
        
        # create dummy file for workerbee
        p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
        open(p_cond + im.path.stem + '.pts', 'a').close() 
    except Exception as e:
        print("The input '{}' failed.".format(p_fr))
        logging.exception("The input '{}' failed.".format(p_fr))
        raise ValueError()

In [ ]:
cats = listdir(path_base)
for cat in cats:
    if not cat[:8] == 'category' or not isdir(path_base + cat):
        warn('Unknown content in path {} (folder: {}).'.format(path_base, cat))
    print(cat)
    # join or create the paths
    p_cat = join(path_base, cat, '')
    p_fr = join(p_cat, 'frames', '')
    p_ln_out_0 = mkdir_p(join(p_cat, fold_out, ''))
    p_bb_out_0 = mkdir_p(join(p_cat, fold_det_out, ''))
    p_condor_dummy_0 = mkdir_p(join(p_cat, 'condor_dummy_' + fold_out, ''))
    assert(isdir(p_fr))  # frames folder should exist
    
    for c in sorted(listdir(p_fr)):   # for each clip
        output_dir = Path(mkdir_p(p_condor_dummy_0 + c + sep))
        done = lambda: output_dir.glob('*.pts')
        im_paths = lambda: mio.image_paths(p_fr + c + '/*')
        exhaust_all_files_randomly(im_paths, done, process_frame, verbose=True)
    rm_if_exists(p_condor_dummy_0)
